In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
import os

root_dir = 'csv_files/Mark Distribution/'

files = [x for x in os.listdir(root_dir) if x[-3:] == 'csv']
dfs = []

for x in files:
    df = pd.read_csv(root_dir + x, index_col=0)
    df['Optimizer'] = x.split('.')[0]
    
    dfs.append(df)
    
df = pd.concat(dfs)

In [3]:
df.head()

,Iterations,Time (sec),Distance,Optimizer
0,200.0,10.214562,0.001794,cs
1,200.0,9.752613,0.010529,cs
2,200.0,7.440755,0.052777,cs
3,200.0,6.808508,0.035303,cs
4,200.0,3.025207,0.021948,cs


In [4]:
optimizers = df['Optimizer'].unique()

In [5]:
grouped_distances = df.groupby("Optimizer")["Distance"]
grouped_time = df.groupby("Optimizer")["Time (sec)"]

times = pd.concat([grouped_time.get_group(x) for x in optimizers], axis=1)
distances = pd.concat([grouped_distances.get_group(x) for x in optimizers], axis=1)

distances.columns = optimizers
times.columns = optimizers

In [6]:
labels = distances.columns

min_distances = pd.DataFrame(labels[np.argmin(distances.values, axis=1)]).value_counts()
max_distances = pd.DataFrame(labels[np.argmax(distances.values, axis=1)]).value_counts()

min_distances.name = "Minimum"
max_distances.name = "Maximum"

pd.concat([min_distances, max_distances], axis=1)

,Minimum,Maximum
0,,
pfw,12,3
cs,11,2
egd,2,20


In [7]:
labels = times.columns

min_times = pd.DataFrame(labels[np.argmin(times.values, axis=1)]).value_counts()
max_times = pd.DataFrame(labels[np.argmax(times.values, axis=1)]).value_counts()

min_times.name = "Minimum"
max_times.name = "Maximum"

pd.concat([min_times, max_times], axis=1)

,Minimum,Maximum
0,,
pfw,19,2
cs,3,5
egd,3,18


In [8]:
print(r"""\begin{tabular}{l|cc|cc|cc}
\toprule
 & \multicolumn{2}{c}{CS}&\multicolumn{2}{c}{EGD} &\multicolumn{2}{c}{PFW} \\
\midrule
Trial & Distance & Time & Distance & Time & Distance & Time \\
\midrule""")

for i in range(len(distances)):
    distances_vals = distances.iloc[i].values
    times_vals = times.iloc[i].values
    
    min_distance_arg, max_distance_arg = np.argmin(distances_vals), np.argmax(distances_vals)
    min_time_arg, max_time_arg = np.argmin(times_vals), np.argmax(times_vals)
    
    s = f"{i + 1} "
    for index, (distance, time) in enumerate(zip(distances_vals, times_vals)):
        # Distance
        if index == min_distance_arg:
            s += r"& \textbf{" + f"{distance:.5f}" + "} "
        elif index == max_distance_arg:
            s += r"& \underline{" + f"{distance:.5f}" + "} "
        else:
            s += f"& {distance:.5f} "
            
        # Time
        if index == min_time_arg:
            s += r"& \textbf{" + f"{time:.2f}" + "} "
        elif index == max_time_arg:
            s += r"& \underline{" + f"{time:.2f}" + "} "
        else:
            s += f"& {time:.2f} "
            
    s += r"\\"
    print(s)
print(r"""\bottomrule
\end{tabular}""")

\begin{tabular}{l|cc|cc|cc}
\toprule
 & \multicolumn{2}{c}{CS}&\multicolumn{2}{c}{EGD} &\multicolumn{2}{c}{PFW} \\
\midrule
Trial & Distance & Time & Distance & Time & Distance & Time \\
\midrule
1 & \textbf{0.00179} & \underline{10.21} & \underline{0.26014} & 9.30 & 0.00442 & \textbf{8.22} \\
2 & \textbf{0.01053} & \underline{9.75} & \underline{0.18086} & \textbf{5.27} & 0.01177 & 7.78 \\
3 & 0.05278 & 7.44 & \underline{0.05282} & \underline{8.50} & \textbf{0.05263} & \textbf{7.04} \\
4 & 0.03530 & 6.81 & \underline{0.03534} & \underline{7.51} & \textbf{0.03528} & \textbf{5.85} \\
5 & \textbf{0.02195} & 3.03 & \underline{0.02210} & \textbf{1.57} & 0.02199 & \underline{5.80} \\
6 & \textbf{0.01742} & 8.20 & 0.01743 & \underline{9.01} & \underline{0.01755} & \textbf{6.60} \\
7 & 0.05822 & 8.97 & \underline{0.05824} & \underline{10.14} & \textbf{0.05814} & \textbf{7.05} \\
8 & \underline{0.03468} & 9.34 & 0.03453 & \underline{9.54} & \textbf{0.03446} & \textbf{7.23} \\
9 & \textbf{0.0070